In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
city = '대구'

path = 'Data/Variants/SIR/alpha_delta/*.csv'
dic_files = Load_files.load_files(path, -18, -16)
data = dic_files[city]
data

,stdDay,Susceptible,Alpha_Infected,Alpha_Recovered,Alpha_Dead,Alpha_alpha,Alpha_beta,Alpha_gamma,Delta_Infected,Delta_Recovered,Delta_Dead,Delta_alpha,Delta_beta,Delta_gamma
0,2021-04-12,2380065,39.65,163.36,2.00,0.059242,0.058260,0.005296,0.32,0.00,0.00,0.564647,0.000000,0.0
1,2021-04-13,2380056,39.47,165.67,2.21,0.072737,0.079807,0.000000,0.50,0.00,0.00,0.441681,0.000000,0.0
2,2021-04-14,2380045,39.18,168.82,2.21,0.119906,0.037519,0.000000,0.72,0.00,0.00,0.501913,0.000000,0.0
3,2021-04-15,2380027,42.39,170.29,2.21,0.024628,0.084218,0.000000,1.08,0.00,0.00,0.074358,0.000000,0.0
4,2021-04-16,2380023,39.86,173.86,2.21,0.078574,0.052684,0.000000,1.16,0.00,0.00,0.207690,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2021-08-11,2376752,12.67,486.64,3.05,0.030942,0.132597,0.000000,814.69,726.09,0.84,0.045715,0.052928,0.0
122,2021-08-12,2376713,11.38,488.32,3.05,0.050350,0.163445,0.000000,808.62,769.21,0.84,0.067316,0.059039,0.0
123,2021-08-13,2376656,10.09,490.18,3.05,0.081696,0.136769,0.000000,815.03,816.95,0.84,0.096082,0.043459,0.0
124,2021-08-14,2376574,9.53,491.56,3.05,0.060128,0.072403,0.000000,857.51,852.37,0.84,0.063482,0.020653,0.0


In [3]:
learning_rates_list = [1e-3, 1e-4]
patiences_list = [20, 50]
num_layers_list = [1, 2, 4]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25]
criterions_list = [nn.MSELoss(), criterion3]
batch_size = 64
hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for hidden_size in hidden_sizes_list:    
                for dropout in dropout_list:
                    for criterion in criterions_list:
                        hyperparameter_dict[i] = [lr, patience, num_layers, hidden_size, dropout, criterion]
                        i += 1

print(i)

72


In [4]:
trained_data = {}
city_list = ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', 
             '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']
models_list = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
               'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']


for city in tqdm(city_list, desc = 'cities', position = 0):
    data = dic_files[city]
    best_model = {}
    
    input_size = 3
    sequence_length = 60
    num_epochs = 10000

    df = Prepare_df.processing(data, 'stdDay', 'Delta_alpha')

    x = df.iloc[:, 0:]
    y = df.iloc[:,:1]

    ms = MinMaxScaler()
    ss = StandardScaler()

    ss.fit(x)
    ms.fit(y)

    for num_model in tqdm(range(12), desc = f'{city}', position = 1):
        rmse_min = np.inf
        h = -1
        
        for hyperparameter in tqdm(hyperparameter_dict.values(), desc = f'{city}_{models_list[num_model]}', position = 2):
            h += 1
            if (num_model < 6) and (h % 2 == 1):
                continue
            model_name = models_list[num_model]
            lr = hyperparameter[0]
            patience = hyperparameter[1]
            num_layers = hyperparameter[2]
            hidden_size = hyperparameter[3]
            dropout = hyperparameter[4]

            if num_model < 6:
                criterion = nn.MSELoss()
            else:
                criterion = hyperparameter[5]

            if num_model < 6:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 50, 60, 1, batch_size, 'mto')
            else:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 37, 60, 14, batch_size, 'mtm')

            if num_model == 0:
                model = RNN(input_size = input_size,
                            hidden_size = hidden_size,
                            sequence_length = sequence_length,
                            num_layers = num_layers, 
                            dropout = dropout, 
                            device = device).to(device)

            elif num_model == 1:
                model = LSTM(input_size = input_size,
                             hidden_size = hidden_size,
                             sequence_length = sequence_length,
                             num_layers = num_layers, 
                             dropout = dropout, 
                             device = device).to(device)

            elif num_model == 2:
                model = GRU(input_size = input_size,
                            hidden_size = hidden_size,
                            sequence_length = sequence_length,
                            num_layers = num_layers, 
                            dropout = dropout, 
                            device = device).to(device)

            elif num_model == 3:
                model = BiRNN(input_size = input_size,
                              hidden_size = hidden_size,
                              sequence_length = sequence_length,
                              num_layers = num_layers, 
                              dropout = dropout, 
                              device = device).to(device)

            elif num_model == 4:
                model = BiLSTM(input_size = input_size,
                               hidden_size = hidden_size,
                               sequence_length = sequence_length,
                               num_layers = num_layers, 
                               dropout = dropout, 
                               device = device).to(device)

            elif num_model == 5:
                model = BiGRU(input_size = input_size,
                              hidden_size = hidden_size,
                              sequence_length = sequence_length,
                              num_layers = num_layers, 
                              dropout = dropout, 
                              device = device).to(device)

            elif num_model == 6:
                model = RNN_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

            elif num_model == 7:
                model = LSTM_encoder_decoder(input_size = input_size, 
                                             hidden_size = hidden_size,
                                             num_layers = num_layers, 
                                             dropout = dropout,
                                             device = device).to(device)

            elif num_model == 8:
                model = GRU_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

            elif num_model == 9:
                model = BiRNN_encoder_decoder(input_size = input_size, 
                                                hidden_size = hidden_size,
                                                num_layers = num_layers, 
                                                dropout = dropout,
                                                device = device).to(device)

            elif num_model == 10:
                model = BiLSTM_encoder_decoder(input_size = input_size, 
                                                  hidden_size = hidden_size,
                                                  num_layers = num_layers, 
                                                  dropout = dropout,
                                                  device = device).to(device)

            elif num_model == 11:
                model = BiGRU_encoder_decoder(input_size = input_size, 
                                                hidden_size = hidden_size,
                                                num_layers = num_layers, 
                                                dropout = dropout,
                                                device = device).to(device)            

            optimizer = Adam(model.parameters(), lr = lr)

            if num_model < 6:
                loss_list, model, epoch = Trainer.Many_to_One(train_loader, 
                                                              test_loader, 
                                                              model, 
                                                              criterion, 
                                                              optimizer, 
                                                              num_epochs, 
                                                              patience,
                                                              device)

                label_y, predicted = predict_mto(model, df, x_ss, y_ms)
                
                if rmse_min > rmse(label_y[-14:], predicted[-14:]):
                    rmse_min = rmse(label_y[-14:], predicted[-14:])
                    best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                          num_layers,hidden_size, dropout]
                
                
            else:
                loss_list, model, epoch = Trainer.Many_to_Many(train_loader, 
                                                               test_loader, 
                                                               model, 
                                                               criterion, 
                                                               optimizer, 
                                                               num_epochs, 
                                                               patience,
                                                               14,
                                                               device)        
                
                label_y, predicted, first_label_y, first_predicted, all_predicted = predict_mtm(model, df, x_ss, y_ms, 
                                                                                                14, 0.5, device)
    
                if rmse_min > rmse(label_y[-14:], predicted[-14:]):
                    rmse_min = rmse(label_y[-14:], predicted[-14:])
                    best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                          num_layers, hidden_size, dropout, criterion]
            
        
            
    trained_data[city] = best_model

cities:   0%|          | 0/17 [00:00<?, ?it/s]

강원:   0%|          | 0/12 [00:00<?, ?it/s]

강원_RNN:   0%|          | 0/72 [00:00<?, ?it/s]

강원_LSTM:   0%|          | 0/72 [00:00<?, ?it/s]

강원_GRU:   0%|          | 0/72 [00:00<?, ?it/s]

...

충북_seq2seq_LSTM:   0%|          | 0/72 [00:00<?, ?it/s]

충북_seq2seq_GRU:   0%|          | 0/72 [00:00<?, ?it/s]

충북_seq2seq_BiRNN:   0%|          | 0/72 [00:00<?, ?it/s]

충북_seq2seq_BiLSTM:   0%|          | 0/72 [00:00<?, ?it/s]

충북_seq2seq_BiGRU:   0%|          | 0/72 [00:00<?, ?it/s]

In [6]:
for city in trained_data:
    for name, values in trained_data[city].items():
        save_model(values[1].state_dict(), f"model/SIR/delta/{city}/{name}.pth")
        save_hyperparameter(values[3:], f"hyperparameter/SIR/delta/{city}/{name}.pkl")